In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from functools import reduce
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

In [63]:
conn = sq.connect('../../../Pipeline-Census-Bureau/Outputs/CensusBureau.db')
sql_query = pd.read_sql('SELECT * FROM [SUSB_Annual]', conn)
data = pd.DataFrame(sql_query)
thelist = ['Cheatham', 'Davidson', 'Dickson', 'Houston', 'Humphreys', 'Montgomery', 'Robertson', 'Rutherford', 
          'Stewart', 'Sumner', 'Trousdale', 'Williamson', 'Wilson']
theotherlist = ['Tennessee', 'United states']
data = data.loc[(data['State Name'].isin(theotherlist)) | (data['County'].isin(thelist))]
thefinallist = ['2019', '2020']
data = data.loc[data['Year'].isin(thefinallist)]
data = data.loc[(data['Enterprise Size'] == '2: <20 employees')&(data['NAICS Description'] == 'Total')]

In [64]:
data.head()

,StateFIPS,State Name,CountyFIPS,County,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year,GEO_ID
14184,47,Tennessee,001,Anderson,--,Total,2: <20 employees,861,870,3797,146123,2020,0500000US47001
14398,47,Tennessee,003,Bedford,--,Total,2: <20 employees,554,555,2315,83133,2020,0500000US47003
14566,47,Tennessee,005,Benton,--,Total,2: <20 employees,195,195,889,27810,2020,0500000US47005
14667,47,Tennessee,007,Bledsoe,--,Total,2: <20 employees,81,81,322,8341,2020,0500000US47007
14720,47,Tennessee,009,Blount,--,Total,2: <20 employees,1493,1499,6717,247194,2020,0500000US47009


In [65]:
conn = sq.connect('../../../Pipeline-Census-Bureau/Outputs/CensusBureau.db')
sql_query = pd.read_sql('SELECT * FROM [SUSB_Annual]', conn)
df = pd.DataFrame(sql_query)
thelist = ['Cheatham', 'Davidson', 'Dickson', 'Houston', 'Humphreys', 'Montgomery', 'Robertson', 'Rutherford', 
          'Stewart', 'Sumner', 'Trousdale', 'Williamson', 'Wilson']
theotherlist = ['Tennessee', 'United States']
df = df.loc[(df['State Name'].isin(theotherlist))&(df['County'].isin(thelist))&
                (df['NAICS Description'] == 'Total')&(df['Enterprise Size'] == '1: Total')]
df = df.loc[(df['Year'] == '2019') | (df['Year'] == '2020')]

In [66]:
df.head()

,StateFIPS,State Name,CountyFIPS,County,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year,GEO_ID
15721,47,Tennessee,021,Cheatham,--,Total,1: Total,589,637,6941,295239,2020,0500000US47021
16719,47,Tennessee,037,Davidson,--,Total,1: Total,16201,20512,470435,26899504,2020,0500000US47037
17328,47,Tennessee,043,Dickson,--,Total,1: Total,940,1017,15059,568684,2020,0500000US47043
20247,47,Tennessee,083,Houston,--,Total,1: Total,97,100,998,32896,2020,0500000US47083
20293,47,Tennessee,085,Humphreys,--,Total,1: Total,313,329,4445,241644,2020,0500000US47085


In [67]:
data = data[['County', 'Establishments', 'Year']].reset_index(drop = True)
data = data.rename(columns = {'Establishments':'Small Business Establishments'})
df = df[['County', 'Establishments', 'Year']].reset_index(drop = True)
df = df.rename(columns = {'Establishments':'Total Establishments'})

In [68]:
data = data.merge(df, on = ['County', 'Year'])
data.head()

,County,Small Business Establishments,Year,Total Establishments
0,Cheatham,470,2020,637
1,Davidson,11905,2020,20512
2,Dickson,658,2020,1017
3,Houston,72,2020,100
4,Humphreys,224,2020,329


In [69]:
cols = ['Small Business Establishments', 'Total Establishments']
data[cols] = data[cols].astype(float)

In [70]:
data['% of Establishments Small Businesses'] = data['Small Business Establishments']/data['Total Establishments']
thelist = ['2019', '2020']
data = data.loc[data['Year'].isin(thelist)]

In [71]:
data

,County,Small Business Establishments,Year,Total Establishments,% of Establishments Small Businesses
0,Cheatham,470.0,2020,637.0,0.737834
1,Davidson,11905.0,2020,20512.0,0.580392
2,Dickson,658.0,2020,1017.0,0.647001
3,Houston,72.0,2020,100.0,0.720000
4,Humphreys,224.0,2020,329.0,0.680851
5,Montgomery,1870.0,2020,3090.0,0.605178
6,Robertson,804.0,2020,1221.0,0.658477
7,Rutherford,3313.0,2020,5631.0,0.588350
8,Stewart,113.0,2020,164.0,0.689024
9,Sumner,2224.0,2020,3391.0,0.655854


In [72]:
data.to_csv('../data/susb.csv', index = False)